In [147]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [76]:
#Cleaning Wdi
wdi = pd.read_csv("/Users/pm/Documents/GitHub/ML_Project_1/data/raw/WDI_Data_t.csv",na_values=["..", "NA", "N/A", ""] )
wdi = wdi.copy()
wdi.head()

,Country Name,Country Code,Series Name,Series Code,2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,Afghanistan,AFG,"Mortality rate, infant (per 1,000 live births)",SP.DYN.IMRT.IN,76.900000,74.000000,71.300000,68.700000,66.400000,64.200000,62.300000,60.400000,58.600000,56.900000,55.300000,53.600000,52.000000,50.400000,NaN
1,Afghanistan,AFG,Access to electricity (% of population),EG.ELC.ACCS.ZS,42.700000,43.200000,69.100000,68.000000,89.500000,71.500000,97.700000,97.700000,93.400000,97.700000,97.700000,97.700000,85.300000,85.300000,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,560.621505,606.694676,651.417134,637.087099,625.054942,565.569730,522.082216,525.469771,491.337221,496.602504,510.787063,356.496214,357.261153,413.757895,NaN
3,Afghanistan,AFG,GDP growth (annual %),NY.GDP.MKTP.KD.ZG,14.362441,0.426355,12.752287,5.600745,2.724543,1.451315,2.260314,2.647003,1.189228,3.911603,-2.351101,-20.738839,-6.240172,2.266944,NaN
4,Afghanistan,AFG,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,8.569672,8.561908,7.897169,8.805964,9.528878,10.105348,11.818590,12.620817,14.208419,14.831320,15.533614,21.508461,23.088169,NaN,NaN


In [77]:
wdi.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2010 [YR2010]', '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]',
       '2014 [YR2014]', '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]',
       '2018 [YR2018]', '2019 [YR2019]', '2020 [YR2020]', '2021 [YR2021]',
       '2022 [YR2022]', '2023 [YR2023]', '2024 [YR2024]'],
      dtype='object')

In [78]:
# drop unnecessary columns
wdi = wdi.drop(columns=["Series Code"])
# rename columns
rename_map =  {
    "Country Name": "country",
    "Country Code": "country_code",
    "Series Name": "indicator"
}
wdi = wdi.rename(columns=rename_map)



In [93]:
# rename year columns and convert to numeric
wdi.columns = wdi.columns.str.replace(r'\s*\[YR\d{4}\]$', '', regex=True)
year_cols = wdi.filter(regex=r'^\d{4}$').columns
wdi[year_cols] = wdi[year_cols].apply(pd.to_numeric, errors='coerce')
wdi.tail(20)

,country,country_code,indicator,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
4241,World,WLD,Access to electricity (% of population),83.447138,84.506233,84.936373,85.710841,86.195801,86.924263,88.104773,88.932088,89.797732,90.108769,90.396088,91.334647,91.289245,91.600137,NaN
4242,World,WLD,GDP per capita (current US$),9530.010809,10471.536428,10570.758813,10744.620364,10911.806738,10175.409420,10221.365980,10765.266223,11326.933902,11376.823341,10963.175642,12418.565925,12819.744838,13263.619549,13673.003958
4243,World,WLD,GDP growth (annual %),4.527147,3.347311,2.707281,2.889022,3.148624,3.134515,2.814361,3.456342,3.296091,2.710040,-2.849082,6.419541,3.359911,2.943333,2.863263
4244,World,WLD,Current health expenditure (% of GDP),9.485347,9.343625,9.351037,9.328072,9.432874,9.710148,9.843793,9.725431,9.647710,9.780088,10.857629,10.339156,9.885794,NaN,NaN
4245,World,WLD,"Literacy rate, adult total (% of people ages 1...",84.227158,84.410782,84.601379,84.903099,85.286392,85.678879,86.000107,86.177002,86.437477,86.547913,86.747452,86.927513,87.155327,87.356003,NaN
4246,World,WLD,"Government expenditure on education, total (% ...",4.094322,4.132525,4.229643,4.347215,4.267350,4.234601,4.172310,4.184885,4.091630,4.137480,4.441360,4.240356,3.774872,NaN,NaN
4247,World,WLD,"School enrollment, primary and secondary (gros...",0.971470,0.973750,0.977350,0.991920,0.994080,0.995040,0.999910,0.998250,0.987030,0.988050,0.985640,NaN,NaN,NaN,NaN
4248,World,WLD,"Hospital beds (per 1,000 people)",2.748307,2.795096,2.886270,2.602031,2.864231,2.867897,2.895183,3.005836,3.051730,3.043725,3.283742,NaN,NaN,NaN,NaN
4249,World,WLD,Current health expenditure per capita (current...,900.176902,974.127467,982.074150,992.234653,1014.944203,973.153997,990.828689,1027.118651,1066.647760,1084.163977,1158.070930,1245.909858,1235.603151,NaN,NaN
4250,World,WLD,"Immunization, DPT (% of children ages 12-23 mo...",83.524147,84.550626,84.319418,83.875508,84.661030,84.837243,85.949918,85.941906,86.630062,86.829246,83.724110,81.893488,84.669944,84.053782,NaN


In [80]:
#convert wide to long
wdi_long = wdi.melt(id_vars=['country', 'country_code', 'indicator'],
          value_vars=year_cols,
          var_name='year',
          value_name='value')

wdi_long.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63915 entries, 0 to 63914
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       63870 non-null  object 
 1   country_code  63840 non-null  object 
 2   indicator     63840 non-null  object 
 3   year          63915 non-null  object 
 4   value         48103 non-null  float64
dtypes: float64(1), object(4)
memory usage: 2.4+ MB


In [ ]:
# Keep only rows with valid ISO3 country codes (3 uppercase letters)
mask_iso3 = wdi_long['country_code'] \
                .astype(str) \
                .str.fullmatch(r'^[A-Z]{3}$')
wdi_long_clean = wdi_long.loc[mask_iso3].copy()


In [121]:
#regional or aggregate data removal
#here we hav a dictionary with the regions and aggregates from the world bank
regions_aggregates_dict = {
    "AFE": "Sub-Saharan Africa (excluding high income)",
    "AFW": "Western Africa",
    "ARB": "Arab World",
    "CSS": "Caribbean small states",
    "CEB": "Central Europe and the Baltics",
    "EAR": "Early-demographic dividend",
    "EAS": "East Asia & Pacific (excluding high income)",
    "EAP": "East Asia & Pacific",
    "TEA": "East Asia & Pacific (IDA & IBRD countries)",
    "EMU": "Euro area",
    "ECS": "Europe & Central Asia (excluding high income)",
    "ECA": "Europe & Central Asia",
    "TEC": "Europe & Central Asia (IDA & IBRD countries)",
    "EUU": "European Union",
    "FCS": "Fragile and conflict affected situations",
    "HPC": "Heavily indebted poor countries (HIPC)",
    "HIC": "High income",
    "IBD": "IBRD only",
    "IBT": "IDA & IBRD total",
    "IDB": "IDA blend",
    "IDX": "IDA total",
    "IDA": "IDA only",
    "LTE": "Late-demographic dividend",
    "LCN": "Latin America & Caribbean (excluding high income)",
    "LAC": "Latin America & Caribbean",
    "TLA": "Latin America & Caribbean (IDA & IBRD countries)",
    "LDC": "Least developed countries: UN classification",
    "LMY": "Low & middle income",
    "LIC": "Low income",
    "LMC": "Lower middle income",
    "MEA": "Middle East & North Africa (excluding high income)",
    "MNA": "Middle East & North Africa",
    "TMN": "Middle East & North Africa (IDA & IBRD countries)",
    "MIC": "Middle income",
    "NAC": "North America",
    "INX": "Not classified",
    "OED": "OECD members",
    "OSS": "Other small states",
    "PSS": "Pacific island small states",
    "PST": "Post-demographic dividend",
    "PRE": "Pre-demographic dividend",
    "SST": "Small states",
    "SAS": "South Asia",
    "TSA": "South Asia (IDA & IBRD countries)",
    "SSF": "Sub-Saharan Africa (excluding high income)",
    "SSA": "Sub-Saharan Africa",
    "TSS": "Sub-Saharan Africa (IDA & IBRD countries)",
    "UMC": "Upper middle income",
    "WLD": "World"
}

# remove rows where 'country_code' is in the regions_aggregates_dict keys 
wdi_long_clean['country_code'] = wdi_long_clean['country_code'].astype(str).str.upper()

[code for code in regions_aggregates_dict.keys()]

wdi_long_clean = wdi_long_clean.loc[mask_region_aggregate].copy()   
agg_codes = set(regions_aggregates_dict)
mask_region_aggregate = ~wdi_long_clean['country_code'].isin(agg_codes)
wdi_long_clean = wdi_long_clean.loc[mask_region_aggregate].copy()

In [122]:
wdi_wide = (
    wdi_long_clean
    .pivot_table(index=['country', 'country_code', 'year'],
                 columns='indicator',
                 values='value')
    .reset_index()
)

# convert columns name to string 
wdi_wide.columns.name = None
wdi_wide = wdi_wide.rename_axis(None, axis=1)

In [123]:
wdi_wide.head()


,country,country_code,year,Access to electricity (% of population),Current health expenditure (% of GDP),Current health expenditure per capita (current US$),"Fertility rate, total (births per woman)",GDP growth (annual %),GDP per capita (current US$),"Government expenditure on education, total (% of GDP)","Hospital beds (per 1,000 people)","Immunization, DPT (% of children ages 12-23 months)","Literacy rate, adult total (% of people ages 15 and above)","Mortality rate, infant (per 1,000 live births)","PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)",People using at least basic drinking water services (% of population),People using at least basic sanitation services (% of population),"Population, female (% of total population)","School enrollment, primary and secondary (gross), gender parity index (GPI)"
0,Afghanistan,AFG,2010,42.7,8.569672,46.424389,6.195,14.362441,560.621505,3.47945,0.43,66.0,NaN,76.9,49.679377,48.826969,35.477471,49.539441,0.62581
1,Afghanistan,AFG,2011,43.2,8.561908,52.187218,6.094,0.426355,606.694676,3.46201,0.45,68.0,31.0,74.0,61.817056,51.472376,37.164572,49.518492,0.66345
2,Afghanistan,AFG,2012,69.1,7.897169,52.452473,5.985,12.752287,651.417134,2.60420,0.47,66.0,NaN,71.3,70.922317,54.149492,38.860305,49.499771,0.66925
3,Afghanistan,AFG,2013,68.0,8.805964,56.160435,5.879,5.600745,637.087099,3.45446,0.46,64.0,NaN,68.7,73.131816,56.858142,40.564852,49.482650,0.65533
4,Afghanistan,AFG,2014,89.5,9.528878,60.049511,5.770,2.724543,625.054942,3.69522,0.46,63.0,NaN,66.4,77.143728,59.598158,42.278403,49.468115,0.65402


In [145]:
wdi_wide["country"].sample(120)

1166           Greenland
1521               Kenya
657              Comoros
1754          Luxembourg
1763    Macao SAR, China
              ...       
822             Djibouti
317              Bermuda
89                Angola
2446               Samoa
1034              France
Name: country, Length: 120, dtype: object

In [116]:
wdi_wide.isna().sum()

country                                                                           0
country_code                                                                      0
year                                                                              0
Access to electricity (% of population)                                         253
Current health expenditure (% of GDP)                                           774
Current health expenditure per capita (current US$)                             776
Fertility rate, total (births per woman)                                        225
GDP growth (annual %)                                                           149
GDP per capita (current US$)                                                    125
Government expenditure on education, total (% of GDP)                          1119
Hospital beds (per 1,000 people)                                               1691
Immunization, DPT (% of children ages 12-23 months)                         

In [148]:
# complete NaN values in other indicators using 'GDP per capita (current US$)' as reference
indicators = [
    'Access to electricity (% of population)',
    'Current health expenditure (% of GDP)',
    'Current health expenditure per capita (current US$)',
    'Fertility rate, total (births per woman)',
    'GDP growth (annual %)',
    'GDP per capita (current US$)',                     #rereference to complete NaN
    'Government expenditure on education, total (% of GDP)',
    'Hospital beds (per 1,000 people)',
    'Immunization, DPT (% of children ages 12-23 months)',
    'Literacy rate, adult total (% of people ages 15 and above)',
    'Mortality rate, infant (per 1,000 live births)',
    'PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)',
    'People using at least basic drinking water services (% of population)',
    'People using at least basic sanitation services (% of population)',
    'Population, female (% of total population)',       # déjà complet, mais on le garde
    'School enrollment, primary and secondary (gross), gender parity index (GPI)'
]

def impute_using_country_and_gdp(df,
                                 cols,
                                 country_col='country_code',
                                 gdp_col='GDP per capita (current US$)'):
    """
    Imputation strategy:
    1. Fill missing values with the median of the same country (across all years).
    2. If a country has no valid data for a given indicator, find another country
       whose GDP per capita is the closest (absolute difference) and use that
       country's median for the indicator.
    Returns the modified DataFrame (in‑place).
    """
    # 1. Pre‑compute medians for each country & indicator 
    country_medians = wdi_wide.groupby(country_col)[cols].median()

    # 2. Compute average GDP per capita for each country (used for similarity)
    gdp_by_country = wdi_wide.groupby(country_col)[gdp_col].mean()

    # 3. Iterate over every indicator 
    for col in cols:
        # 1) Fill with own‑country median 
        mask_missing = df[col].isna()
        wdi_wide.loc[mask_missing, col] = wdi_wide.loc[mask_missing, country_col].map(
            country_medians[col]
        )

        # 2) Still missing? -> find closest‑GDP country 
        still_missing = df[col].isna()
        if still_missing.any():
            # Unique list of countries that still need a value for this column
            countries_to_fill = wdi_wide.loc[still_missing, country_col].unique()

            for target in countries_to_fill:
                target_gdp = gdp_by_country.get(target, np.nan)
                if np.isnan(target_gdp):
                    # No GDP info for the target country – cannot match
                    continue

                # Candidates are countries that have a *non‑nan* median for this column
                candidates = country_medians[col].dropna().index
                # Remove the target itself (just in case it got a median later)
                candidates = [c for c in candidates if c != target]

                if not candidates:
                    continue  # No suitable donor country

                # Get GDP values for the candidate pool
                candidate_gdps = gdp_by_country.loc[candidates]
                candidate_gdps = candidate_gdps.dropna()
                if candidate_gdps.empty:
                    continue  # All candidates lack GDP data

                # Choose the country with the smallest absolute GDP difference
                closest = (candidate_gdps - target_gdp).abs().idxmin()

                # Median value from the closest‑GDP country
                fill_value = country_medians.at[closest, col]

                # Apply the fill value to every missing row of the target country
                df.loc[
                    (df[country_col] == target) & (df[col].isna()),
                    col
                ] = fill_value

    return wdi_wide


wdi_wide = impute_using_country_and_gdp(
    df=wdi_wide,
    cols=indicators,
    country_col='country_code',
    gdp_col='GDP per capita (current US$)'
)
#Verify that no NaNs remain in the indicator columns
print("\nRemaining NaNs after imputation:")
print(wdi_wide[indicators].isna().sum())


Remaining NaNs after imputation:
Access to electricity (% of population)                                         0
Current health expenditure (% of GDP)                                          45
Current health expenditure per capita (current US$)                            45
Fertility rate, total (births per woman)                                        0
GDP growth (annual %)                                                          45
GDP per capita (current US$)                                                   45
Government expenditure on education, total (% of GDP)                          30
Hospital beds (per 1,000 people)                                               30
Immunization, DPT (% of children ages 12-23 months)                            30
Literacy rate, adult total (% of people ages 15 and above)                     30
Mortality rate, infant (per 1,000 live births)                                 15
PM2.5 air pollution, mean annual exposure (micrograms per cubic 

In [151]:

wdi_wide.head()


,country,country_code,year,Access to electricity (% of population),Current health expenditure (% of GDP),Current health expenditure per capita (current US$),"Fertility rate, total (births per woman)",GDP growth (annual %),GDP per capita (current US$),"Government expenditure on education, total (% of GDP)","Hospital beds (per 1,000 people)","Immunization, DPT (% of children ages 12-23 months)","Literacy rate, adult total (% of people ages 15 and above)","Mortality rate, infant (per 1,000 live births)","PM2.5 air pollution, mean annual exposure (micrograms per cubic meter)",People using at least basic drinking water services (% of population),People using at least basic sanitation services (% of population),"Population, female (% of total population)","School enrollment, primary and secondary (gross), gender parity index (GPI)"
0,Afghanistan,AFG,2010,42.7,8.569672,46.424389,6.195,14.362441,560.621505,3.47945,0.43,66.0,33.753841,76.9,49.679377,48.826969,35.477471,49.539441,0.62581
1,Afghanistan,AFG,2011,43.2,8.561908,52.187218,6.094,0.426355,606.694676,3.46201,0.45,68.0,31.000000,74.0,61.817056,51.472376,37.164572,49.518492,0.66345
2,Afghanistan,AFG,2012,69.1,7.897169,52.452473,5.985,12.752287,651.417134,2.60420,0.47,66.0,33.753841,71.3,70.922317,54.149492,38.860305,49.499771,0.66925
3,Afghanistan,AFG,2013,68.0,8.805964,56.160435,5.879,5.600745,637.087099,3.45446,0.46,64.0,33.753841,68.7,73.131816,56.858142,40.564852,49.482650,0.65533
4,Afghanistan,AFG,2014,89.5,9.528878,60.049511,5.770,2.724543,625.054942,3.69522,0.46,63.0,33.753841,66.4,77.143728,59.598158,42.278403,49.468115,0.65402


In [154]:


wdi_wide.to_csv('df_wdi.csv', index=False)